http://brandonrose.org/clustering#Tf-idf-and-document-similarity

In [2]:
import pandas as pd
import numpy as np
import collections
import json

In [3]:
lda_result = pd.read_csv('./data_output/lda.tsv', sep='\t')
km_result = pd.read_csv('./data_output/km.tsv', sep='\t')
dec_result = pd.read_csv('./data_output/dec.tsv', sep='\t')

In [4]:
lda_result.head()

,id,document,topic,topic_prob,topic_word,plot_x,plot_y,td_x,td_y,td_z
0,0,Hazard Ratio (and 95% Confidence Intervals) ...,0,16,"regress(33),model(31),hazard(24),ventricular(2...",9.664474,8.992035,103.587460,72.253586,5.656330
1,1,Left Ventricular and Right Ventricular Ejectio...,0,25,"regress(33),model(31),hazard(24),ventricular(2...",23.601093,-15.201798,31.781635,89.926690,-82.635830
2,2,Partial Regression Coefficients (All Subjects...,0,12,"regress(33),model(31),hazard(24),ventricular(2...",11.646137,0.977639,20.661259,124.873360,-45.061954
3,3,Left Ventricular (LVEF) and Right Ventricula...,0,31,"regress(33),model(31),hazard(24),ventricular(2...",23.601820,-15.202716,24.231483,92.084380,-102.178580
4,4,Predictors of Mortality by Multivariable Analy...,0,11,"regress(33),model(31),hazard(24),ventricular(2...",8.242556,3.281180,-146.370940,8.817643,-81.400190


In [5]:
km_result.head()

,id,document,topic,plot_x,plot_y,topic_word,words_count
0,0,Hazard Ratio (and 95% Confidence Intervals) ...,7,9.664474,8.992035,"[('regress', '4.34'), ('model', '3.90'), ('ana...",19
1,1,Left Ventricular and Right Ventricular Ejectio...,3,23.601093,-15.201798,"[('coronari', '4.33'), ('valu', '1.96'), ('sen...",25
2,2,Partial Regression Coefficients (All Subjects...,7,11.646137,0.977639,"[('regress', '4.34'), ('model', '3.90'), ('ana...",15
3,3,Left Ventricular (LVEF) and Right Ventricula...,3,23.601820,-15.202716,"[('coronari', '4.33'), ('valu', '1.96'), ('sen...",31
4,4,Predictors of Mortality by Multivariable Analy...,7,8.242556,3.281180,"[('regress', '4.34'), ('model', '3.90'), ('ana...",11


In [6]:
dec_result.head()

,id,document,topic,plot_x,plot_y
0,0,Hazard Ratio (and 95% Confidence Intervals) ...,4,9.664474,8.992035
1,1,Left Ventricular and Right Ventricular Ejectio...,7,23.601093,-15.201798
2,2,Partial Regression Coefficients (All Subjects...,7,11.646137,0.977639
3,3,Left Ventricular (LVEF) and Right Ventricula...,7,23.601820,-15.202716
4,4,Predictors of Mortality by Multivariable Analy...,7,8.242556,3.281180


In [3]:
# km_result = pd.DataFrame([0, 6, 0, 6, 6, 6, 6, 0, 0, 6, 0, 0, 0, 6, 0, 6, 6, 6, 6, 0, 0, 0,
#        0, 0, 2, 6, 6, 0, 6, 6, 2, 0, 0, 0, 0, 6, 4, 2, 6, 7, 0, 6, 6, 0,
#        5, 0, 6, 6, 6, 2, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 0, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3,
#        3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 3, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#        2, 1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
#        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 6, 2, 2, 2, 2,
#        6, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2, 2, 2, 2, 2, 2, 2, 6, 2, 2, 2,
#        2, 2, 6, 2, 6, 2, 2, 2, 2, 4, 2, 2, 6, 2, 2, 2, 6, 2, 2, 2, 2, 2,
#        4, 2, 4, 4, 4, 4, 0, 7, 7, 7, 4, 4, 0, 2, 4, 0, 7, 7, 4, 7, 0, 4,
#        4, 2, 4, 7, 0, 7, 0, 0, 0, 4, 4, 2, 7, 0, 4, 4, 4, 0, 0, 4, 4, 0,
#        4, 0, 4, 4, 4, 4, 2, 4, 5, 6, 6, 6, 5, 5, 6, 5, 5, 5, 5, 5, 6, 5,
#        6, 5, 7, 5, 6, 7, 5, 0, 0, 6, 6, 6, 6, 5, 5, 5, 0, 6, 6, 6, 6, 2,
#        7, 7, 5, 5, 7, 5, 6, 2, 6, 6, 7, 0, 5, 7, 0, 0, 0, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 6, 0, 0, 0, 0, 0,
#        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
#        4, 0, 0, 0, 4, 4, 7, 7, 0, 7, 7, 2, 7, 7, 7, 2, 7, 7, 2, 7, 7, 2,
#        7, 7, 7, 7, 7, 7, 7, 7, 0, 7, 7, 2, 7, 7, 7, 6, 7, 2, 7, 7, 7, 7,
#        7, 7, 7, 7], columns=['topic']).reset_index()
# km_result.columns = ['id', 'topic']

In [12]:
def intersect(a, b):
    return len(list(set(a) & set(b)))

In [7]:
lda_docs_by_topic = lda_result.groupby('topic').agg({'id': 'unique'})
km_docs_by_topic = km_result.groupby('topic').agg({'id': 'unique'})
dec_docs_by_topic = dec_result.groupby('topic').agg({'id': 'unique'})

In [8]:
lda_docs_by_topic.head()

,id
topic,
0,"[0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13,..."
1,"[50, 52, 53, 54, 55, 60, 62, 64, 65, 68, 70, 7..."
2,"[51, 56, 57, 58, 59, 61, 63, 67, 73, 74, 84, 2..."
3,"[69, 163, 251, 256, 280, 282, 283, 289, 296, 2..."
4,"[195, 201, 202, 207, 209, 210, 211, 214, 216, ..."


In [9]:
km_docs_by_topic.head()

,id
topic,
0,"[25, 50, 51, 53, 54, 60, 63, 65, 66, 69, 75, 7..."
1,"[101, 102, 103, 104, 105, 106, 107, 108, 109, ..."
2,"[350, 351, 352, 353, 354, 355, 356, 357, 358, ..."
3,"[1, 3, 8, 10, 14, 31, 35, 36, 154, 155, 161, 2..."
4,"[83, 87, 91, 96, 195, 200, 201, 202, 203, 204,..."


In [10]:
dec_docs_by_topic.head()

,id
topic,
0,"[63, 85, 105, 109, 112, 116, 118, 119, 135, 14..."
1,"[20, 101, 103, 106, 123, 124, 129, 131, 137, 1..."
2,"[120, 234, 350, 351, 352, 353, 354, 355, 356, ..."
3,"[44, 54, 59, 67, 74, 178, 187, 259, 267]"
4,"[0, 5, 6, 7, 8, 13, 15, 16, 17, 19, 22, 23, 27..."


In [20]:
[intersect(km_docs_by_topic.iloc[4].values[0], lda_docs_by_topic.iloc[j].values[0]) for j in range(5)]

[1, 4, 19, 0, 30]

In [9]:
lda_km_topic_map = {i: [] for i in range(5)} # key: lda, value: km
km_lda_topic_map = {i: [] for i in range(5)} # key: km, value: lda

for i in range(5):
    lda_topic = int(np.argmax([intersect(km_docs_by_topic.iloc[i].values[0], lda_docs_by_topic.iloc[j].values[0]) for j in range(5)]))
    km_topic = i
    lda_km_topic_map[lda_topic].append(km_topic)
    km_lda_topic_map[km_topic].append(lda_topic)

In [10]:
lda_km_topic_map, km_lda_topic_map

({0: [7], 1: [0, 5], 2: [], 3: [2], 4: [4], 5: [3], 6: [1], 7: [6]},
 {0: [1], 1: [6], 2: [3], 3: [5], 4: [4], 5: [1], 6: [7], 7: [0]})

In [11]:
lda_dec_topic_map = {i: [] for i in range(8)} # key: lda, value: km
dec_lda_topic_map = {i: [] for i in range(8)} # key: km, value: lda

for i in range(8):
    lda_topic = int(np.argmax([intersect(dec_docs_by_topic.iloc[i].values[0], lda_docs_by_topic.iloc[j].values[0]) for j in range(8)]))
    dec_topic = i
    lda_dec_topic_map[lda_topic].append(dec_topic)
    dec_lda_topic_map[dec_topic].append(lda_topic)

In [12]:
lda_dec_topic_map, dec_lda_topic_map

({0: [4], 1: [], 2: [], 3: [2], 4: [], 5: [7], 6: [0, 1, 5], 7: [3, 6]},
 {0: [6], 1: [6], 2: [3], 3: [7], 4: [0], 5: [6], 6: [7], 7: [5]})

In [13]:
merged_result = pd.merge(lda_result[['id', 'document', 'topic']], km_result[['id', 'topic']], on='id', suffixes=('_lda', '_km'))
merged_result = pd.merge(merged_result[['id', 'document', 'topic_lda', 'topic_km']], dec_result[['id', 'topic']], on='id')
merged_result.columns = ['id', 'document', 'topic_lda', 'topic_km', 'topic_dec']

In [14]:
merged_result.head()

,id,document,topic_lda,topic_km,topic_dec
0,0,Hazard Ratio (and 95% Confidence Intervals) ...,0,7,4
1,1,Left Ventricular and Right Ventricular Ejectio...,0,3,7
2,2,Partial Regression Coefficients (All Subjects...,0,7,7
3,3,Left Ventricular (LVEF) and Right Ventricula...,0,3,7
4,4,Predictors of Mortality by Multivariable Analy...,0,7,7


In [15]:
json_data = collections.OrderedDict()
json_data['lda_km_topic_map'] = lda_km_topic_map
json_data['km_lda_topic_map'] = km_lda_topic_map
json_data['lda_dec_topic_map'] = lda_dec_topic_map
json_data['dec_lda_topic_map'] = dec_lda_topic_map
json_data['rows'] = merged_result[['document', 'topic_lda', 'topic_km', 'topic_dec']].to_dict(orient='records')
jsonp = json.loads(json.dumps(json_data, ensure_ascii=False, indent='\t').replace('`', ''))

In [16]:
f = open('./Visualization/res/document_table.json', 'w')
f.write(json.dumps(json_data, ensure_ascii=False, indent='\t').replace('`', ''))
f.close()